# Ground-level Analysis - Schools

Part of the iRAP road star rating system includes two attributes for school zones - one for signage/lights for school zones and another for crossing supervisors.

For school signage/lights we can:

1. Find geographic locations of schools (for ex. https://nces.ed.gov/programs/edge/Geographic/SchoolLocations)
1. Find geographic locations of roads around those schools.
1. Seek streetview imagery around the schools using geographic location data (for ex. Google Streetview, etc)
1. Use machine-intelligent techniques to surmise whether there are school zone signs or specialized lights (per region/country as this is typically standardized)
1. Estimate the iRAP code for the "School zone warning" attribute for road segments

An indirect advantage of this might be that roads which are not near schools could be noted to have a code of "4", or not applicable. There may also be a way to determine if there are crossing supervisors though I imagine this will be much more difficult (I've personally noticed they don't stick around all the time so streetview data may not be an accurate depiction).

## To-do

- Find streetview images using data produced from Task 7
- Use machine-intelligent methods to surmise whether there are school signs in the images from the streetviews of roads around schools.
- Determine coding for roads to provide score for road segment based on findings.

## References

- iRAP
    - Coding Manual - Section "School zone warning" - http://resources.irap.org/Specifications/iRAP_Coding_Manual_Drive_on_Left.pdf
    - Code Upload Specification (code details) - Column "School Zone Warning" - http://resources.irap.org/Specifications/RAP-SR-3-3_Upload_file_specification.xlsx
    - Global KPI Specification - https://www.starratingforschools.org/
        - Star Rating for Schools Coding Specification - https://downloads.starratingforschools.org/Toolkit/SR4S_Coding_Guide.pdf
- School Zones - https://en.wikipedia.org/wiki/School_zone
    - US Signage Details https://mutcd.fhwa.dot.gov/htm/2009r1r2/part7/part7b.htm#section7B08
    - US Signage example standards https://www.state.nj.us/transportation/community/srts/pdf/szdgchapter3.pdf
- NCES
    - School location data https://nces.ed.gov/programs/edge/Geographic/SchoolL
- Google Streetview
    - Overview https://developers.google.com/maps/documentation/streetview/overview
    - Pricing https://developers.google.com/maps/documentation/streetview/usage-and-billing
    - API Config Page https://console.cloud.google.com/apis/library/street-view-image-backend.googleapis.com
    - API Billing Enable page https://console.cloud.google.com/projectselector2/billing/enable
    - API reference libs
        - https://github.com/rrwen/google_streetview
        - https://github.com/robolyst/streetview
- OpenStreetCam (part of OSM)
    - https://openstreetcam.org/map/
    - API Docs http://api.openstreetcam.org/api/doc.html
    - Terms https://openstreetcam.org/terms
    - Change in name to KartaView http://blog.improveosm.org/en/2020/11/hello-kartaview/
    - KartaView
        - http://kartaview.org/map
        - Overview http://kartaview.org/landing
        - Plugin seemingly using signage detection http://blog.improveosm.org/en/2020/03/openstreetcam-plug-in-search-box-for-detection-filtering/
        - Plugin source https://github.com/kartaview/josm-plugin
        - Upload scripts client for OSC https://github.com/kartaview/upload-scripts
        - Sample request gathered from web: http://api.openstreetcam.org/2.0/photo/?lat=34.94083565649461&lng=-82.8662492514243&zoomLevel=13
- Bing Maps Streetside
    - Overview https://www.microsoft.com/en-us/maps/streetside
    - API reference lib https://github.com/ylt/Bing-Streetside
    - API example https://docs.microsoft.com/en-us/bingmaps/articles/getting-streetside-tiles-from-imagery-metadata
    - Licensing for free, educational, or non-profit tier as "Developer" https://www.microsoft.com/en-us/maps/licensing#developerLicense
    - Key registration process https://docs.microsoft.com/en-us/bingmaps/getting-started/bing-maps-dev-center-help/getting-a-bing-maps-key
- Mapillary
    - https://www.mapillary.com/dataset/trafficsign
    - Feature detection details https://help.mapillary.com/hc/en-us/articles/115002332165
    - Signage https://www.mapillary.com/developer/api-documentation/#warning
    - Sign up page https://www.mapillary.com/app/?signup=true&lat=20&lng=0&z=1.5
    - API notes https://blog.mapillary.com/update/2020/08/28/map-data-mapillary-api.html
    - APR reference lib https://github.com/khmurakami/pymapillary
- Road Signage Datasets
    - LISA Traffic Sign Dataset (USA) http://cvrr.ucsd.edu/LISA/lisa-traffic-sign-dataset.html
    - Kaggle Traffic Sign Competition Dataset https://www.kaggle.com/c/traffic-sign-recognition
    - Belgian Traffic Sign Dataset https://btsd.ethz.ch/shareddata/
    - German Traffic Sign Dataset https://benchmark.ini.rub.de/?section=gtsdb&subsection=dataset
    - Google Open Images Dataset https://storage.googleapis.com/openimages/web/index.html
        - Google Traffic Signs https://storage.googleapis.com/openimages/web/visualizer/index.html?set=train&type=detection&c=%2Fm%2F01mqdt
    - Wikimedia US Roadsigns https://commons.wikimedia.org/wiki/Road_signs_of_the_United_States#Schools
    

In [24]:
import google_streetview.api
import google_streetview.helpers
import pandas as pd
from IPython.display import Image
from box import Box
import requests
import os

from osc_client.osc_client import OSC_Client

# assumes existing config.yaml file with format: 
# google:
#     key: "api_key_here"
# mapillary:
#     key: "api_key_here"
config = Box.from_yaml(filename="../../../config.yaml")

In [28]:
school_roads = pd.read_csv("../../task7-feature-extraction-using-aerial-level-data/data/school_roads-sample.csv")
school_roads.head()

,school_NCESSCH,school_LEAID,school_street,school_city,school_state,school_country,school,school_lat,school_lon,road_name,lat,lon,road_length,bearing
0,10000500870,100005,600 E Alabama Ave,Albertville,AL,USA,Albertville Middle School,34.260194,-86.206174,East Alabama Avenue,34.259944,-86.207459,112.473,184.046950
1,10000500870,100005,600 E Alabama Ave,Albertville,AL,USA,Albertville Middle School,34.260194,-86.206174,East Alabama Avenue,34.260201,-86.207437,112.473,184.046950
2,10000500870,100005,600 E Alabama Ave,Albertville,AL,USA,Albertville Middle School,34.260194,-86.206174,East Alabama Avenue,34.260555,-86.207397,112.473,185.335301
3,10000500870,100005,600 E Alabama Ave,Albertville,AL,USA,Albertville Middle School,34.260194,-86.206174,East Alabama Avenue,34.260952,-86.207358,112.473,184.641739
4,10000500870,100005,600 E Alabama Ave,Albertville,AL,USA,Albertville Middle School,34.260194,-86.206174,Davis Street,34.259944,-86.207459,254.561,91.459085


In [33]:
school_roads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1042 entries, 0 to 1041
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   school_NCESSCH  1042 non-null   int64  
 1   school_LEAID    1042 non-null   int64  
 2   school_street   1042 non-null   object 
 3   school_city     1042 non-null   object 
 4   school_state    1042 non-null   object 
 5   school_country  1042 non-null   object 
 6   school          1042 non-null   object 
 7   school_lat      1042 non-null   float64
 8   school_lon      1042 non-null   float64
 9   road_name       910 non-null    object 
 10  lat             1042 non-null   float64
 11  lon             1042 non-null   float64
 12  road_length     1042 non-null   float64
 13  bearing         1042 non-null   float64
dtypes: float64(6), int64(2), object(6)
memory usage: 114.1+ KB


## OpenStreetCam Analysis

In [9]:
# instantiate our client and test a random lat,lng point with osc photos available
osc_client = OSC_Client()
photos_list = osc_client.get_photos(lat=34.94083565649461, lng=-82.8662492514243)
photos_list[0]

[{osc_client.py:39} INFO - Requesting OSC photos from lat,lng point: 34.94083565649461, -82.8662492514243


{'autoImgProcessingResult': 'BLURRED',
 'autoImgProcessingStatus': 'FINISHED',
 'cameraParameters': None,
 'dateAdded': '2018-12-16 17:20:37',
 'dateProcessed': '2018-12-16 17:21:50',
 'distance': '27.81',
 'fieldOfView': None,
 'filepath': '2018/12/16/{{sizeprefix}}/1315139_711d7_4194.jpg',
 'filepathLTh': '2018/12/16/lth/1315139_711d7_4194.jpg',
 'filepathProc': '2018/12/16/proc/1315139_711d7_4194.jpg',
 'filepathTh': '2018/12/16/th/1315139_711d7_4194.jpg',
 'fileurl': 'https://storage9.openstreetcam.org/files/photo/2018/12/16/{{sizeprefix}}/1315139_711d7_4194.jpg',
 'fileurlLTh': 'https://storage9.openstreetcam.org/files/photo/2018/12/16/lth/1315139_711d7_4194.jpg',
 'fileurlProc': 'https://storage9.openstreetcam.org/files/photo/2018/12/16/proc/1315139_711d7_4194.jpg',
 'fileurlTh': 'https://storage9.openstreetcam.org/files/photo/2018/12/16/th/1315139_711d7_4194.jpg',
 'from': '113834146',
 'gpsAccuracy': '9.0000',
 'hasObd': '1',
 'heading': '232',
 'height': '1728',
 'id': '337996

In [5]:
Image(url=photos_list[1]["fileurlProc"])

In [6]:
photo_results = []
for index, row in school_roads.iterrows():
    # break our results after 50 queries just for testing
    if index > 50:
        break
    try:
        photos_list = osc_client.get_photos(lat=row["lat"], lng=row["lon"])
        photo_results.append(photos_list)
    except:
        pass

print(photo_results)

[{<ipython-input-3-1ed9d077fdc7>:39} INFO - Requesting OSC photos from lat,lng point: 34.259944, -86.207459
[{<ipython-input-3-1ed9d077fdc7>:39} INFO - Requesting OSC photos from lat,lng point: 34.260201, -86.207437
[{<ipython-input-3-1ed9d077fdc7>:39} INFO - Requesting OSC photos from lat,lng point: 34.260555, -86.207397
[{<ipython-input-3-1ed9d077fdc7>:39} INFO - Requesting OSC photos from lat,lng point: 34.260952, -86.207358
[{<ipython-input-3-1ed9d077fdc7>:39} INFO - Requesting OSC photos from lat,lng point: 34.259944, -86.207459
[{<ipython-input-3-1ed9d077fdc7>:39} INFO - Requesting OSC photos from lat,lng point: 34.259948, -86.207649
[{<ipython-input-3-1ed9d077fdc7>:39} INFO - Requesting OSC photos from lat,lng point: 34.259972999999995, -86.208069
[{<ipython-input-3-1ed9d077fdc7>:39} INFO - Requesting OSC photos from lat,lng point: 34.259980999999996, -86.20841
[{<ipython-input-3-1ed9d077fdc7>:39} INFO - Requesting OSC photos from lat,lng point: 34.259977, -86.209011
[{<ipython-

## Google Streetview Analysis

In [29]:
# attempting to recreate openstreetcam.org image from above
params = [{
    'size': '640x640', # max 640x640 pixels
    'location': '34.94083565649461,-82.8662492514243',
    'heading': "90",
    'key': config.google.key
}]

# Create a results object
results = google_streetview.api.results(params)

In [30]:
#Image(url=results.links[0])

In [31]:
results.metadata[0]["status"]

'OK'

In [32]:
# attempt to gather gstreetview meta
photo_results = []
for index, row in school_roads.iterrows():
    # break our results after 200 queries just for testing
    if index > 200:
        break
    try:
        if index > 50:
            params = [{
                'size': '640x640', # max 640x640 pixels
                'location': "{lat},{lon}".format(lat=row["lat"],lon=row["lon"]),
                'heading': str(row["bearing"]),
                'key': config.google.key
            }]

            # Create a results object
            results = google_streetview.api.results(params)

            # only append to list if result status was ok
            if results.metadata[0]["status"] == 'OK':
                photo_results.append(results)
    except:
        pass

len(photo_results)

150

In [34]:
# make a data dir for dumping the image downloads to.
if not os.path.isdir("{}/data".format(os.getcwd())):
    os.mkdir("{}/data".format(os.getcwd()))

In [35]:
# download images locally
for result in photo_results:
    url = result.links[0]
    
    # use filename format lat_lon_bearing.jpg
    filename = "{}_{}.jpg".format(result.params[0]["location"].replace(",","_"),
                   result.params[0]["heading"])
    filepath = "{}/data/{}".format(os.getcwd(),filename)
    
    # get image from url
    img_data = requests.get(url).content
    
    # write image to filepath
    with open(filepath, 'wb') as handler:
        handler.write(img_data)

## Mapillary Location Objects

In [11]:
from pymapillary import Mapillary
from pymapillary.utils import *

# Every parameter that can be passed in to this search function
bbox = "16.430300,7.241686,16.438757,7.253186" # minx,miny,maxx,maxy
sample_point = "-82.8662492514243, 34.94083565649461"
per_page = 200

# Create a Mapillary Object
map = Mapillary(config.mapillary.key)
raw_json = map.search_images(closeto=sample_point, per_page=per_page)
raw_json


{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'ca': 55.30157470703125,
    'camera_make': 'Apple',
    'camera_model': 'iPhone',
    'captured_at': '2018-03-20T23:29:52.150Z',
    'key': 'c9QqbfR5MhGkBwmCykqopg',
    'pano': False,
    'sequence_key': 'r3yoz4942vgdilcaw4ycdn',
    'user_key': 'nVYd2P677TaeNb7dQgzSvQ',
    'username': 'i_drive_alot',
    'quality_score': 4},
   'geometry': {'type': 'Point', 'coordinates': [-82.8664161, 34.9408464]}}]}

In [12]:
raw_json = map.search_map_features(closeto=sample_point, per_page=per_page)
raw_json

{'type': 'FeatureCollection', 'features': []}

## ImageAI Out-of-box Predictions

In [3]:
from imageai.Prediction import ImagePrediction
import os
execution_path = os.getcwd()

In [28]:
multiple_prediction = ImagePrediction()
multiple_prediction.setModelTypeAsResNet()
multiple_prediction.setModelPath(os.path.join(execution_path, "resnet50_weights_tf_dim_ordering_tf_kernels.h5"))
multiple_prediction.loadModel()

all_images_array = []

all_files = os.listdir("{}/data".format(execution_path))
for each_file in all_files:
    if(each_file.endswith(".jpg") or each_file.endswith(".png")):
        all_images_array.append("{}/data/{}".format(execution_path, each_file))

results_array = multiple_prediction.predictMultipleImages(all_images_array, result_count_per_image=5)

for index_i in range(len(results_array)):
    predictions, percentage_probabilities = results_array[index_i]["predictions"], results_array[index_i]["percentage_probabilities"]
    print(os.path.basename(all_images_array[index_i]))
    for index_x in range(len(predictions)):
        print(predictions[index_x] , " : " , percentage_probabilities[index_x])
    print("-----------------------")

34.2651_-86.20527299999999_326.0438192409169.jpg
trailer_truck  :  14.387580752372742
beacon  :  5.928321182727814
horse_cart  :  5.919157713651657
water_tower  :  5.467585101723671
mailbox  :  3.0199337750673294
-----------------------
34.262795000000004_-86.20737199999999_335.16438456722716.jpg
traffic_light  :  11.400825530290604
trailer_truck  :  10.914980620145798
unicycle  :  8.150561898946762
pole  :  6.010798364877701
freight_car  :  5.593433976173401
-----------------------
34.264469_-86.20673000000001_238.51894790071773.jpg
water_tower  :  6.59431517124176
wreck  :  4.886839911341667
pole  :  4.8547327518463135
flagpole  :  3.2283391803503036
electric_locomotive  :  2.887953631579876
-----------------------
34.263126_-86.20756999999999_329.4046842496186.jpg
freight_car  :  10.88649034500122
steam_locomotive  :  7.494565844535828
chainlink_fence  :  5.374998599290848
obelisk  :  4.9683548510074615
streetcar  :  4.9089450389146805
-----------------------
34.25996_-86.209965_89.

In [23]:
# custom training https://imageai.readthedocs.io/en/latest/custom/index.html

from imageai.Prediction.Custom import ModelTraining

model_trainer = ModelTraining()

model_trainer.setModelTypeAsResNet()

model_trainer.setDataDirectory(data_directory="{}/../data/schools/imageai".format(execution_path))

In [ ]:
model_trainer.trainModel(num_objects=1,
                         num_experiments=100,
                         enhance_data=True,
                         batch_size=1,
                         show_network_summary=False)